In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

## Cleaning the test set lyrics and creating One-Hots for train data's genres and just cleaned lyrics

In [6]:
lyrics, word_list = clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
train_genres = one_hot_genres(train_df['Genre'])

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [8]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list))
print(train_lyrics.shape)
lyrics_size = len(word_list)

# Cleaning no more used variables
del train_df

(12240, 37818)


## Cleaning the test set lyrics and creating One-Hots for test data's genres and just cleaned lyrics

In [9]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_list = clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [10]:
test_genres = one_hot_genres(test_df['Genre'])

In [11]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size)
print(test_lyrics.shape)

# Cleaning no more used variables
del test_df

(2160, 37818)


# Neural Network stuff 
## **Modify and run only the cells below to change the neural network**

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(units=128, input_shape=(lyrics_size,)))
model.add(Activation('relu'))            # activation layer
model.add(Dropout(0.8))
model.add(Dense(20))
model.add(Dense(8))
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               4840832   
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)                2580      
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 168       
_________________________________________________________________
activation_7 (Activation)    (None, 8)                 0         
Total params: 4,843,580
Trainable params: 4,843,580
Non-trainable params: 0
____________________________________________

In [30]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.0001) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(train_lyrics, train_genres, epochs=10, validation_split=17/100, batch_size=16);

Train on 10159 samples, validate on 2081 samples
Epoch 1/15
10159/10159 [==============================] - 9s 842us/sample - loss: 1.9445 - acc: 0.2492 - val_loss: 1.7696 - val_acc: 0.4190
Epoch 2/15
10159/10159 [==============================] - 9s 869us/sample - loss: 1.6894 - acc: 0.3946 - val_loss: 1.6039 - val_acc: 0.4613
Epoch 3/15
10159/10159 [==============================] - 8s 832us/sample - loss: 1.5129 - acc: 0.4695 - val_loss: 1.5078 - val_acc: 0.4767
Epoch 4/15
10159/10159 [==============================] - 9s 840us/sample - loss: 1.3733 - acc: 0.5259 - val_loss: 1.4428 - val_acc: 0.4926
Epoch 5/15
10159/10159 [==============================] - 8s 823us/sample - loss: 1.2633 - acc: 0.5693 - val_loss: 1.4007 - val_acc: 0.5036
Epoch 6/15
10159/10159 [==============================] - 8s 827us/sample - loss: 1.1828 - acc: 0.6046 - val_loss: 1.3721 - val_acc: 0.5103
Epoch 7/15
10159/10159 [==============================] - 8s 831us/sample - loss: 1.1015 - acc: 0.6312 - val_lo

## Testing the Neural Network's performace + Confusion Matrix

In [32]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)

2160/2160 [==============================] - 1s 250us/sample - loss: 1.3931 - acc: 0.5241


In [16]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(test_lyrics)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

[[101  37  16  31  31  36  14  26]
 [ 23 108   1  33   9  50  23  30]
 [  8   0 226   0  22   1   5   0]
 [ 13  20   0 189   9   5   9  42]
 [ 19   9  30  20 125   8  17   9]
 [  8  36   2   5   3 172  14  21]
 [ 17  32  24  15  34  28  88  25]
 [ 20  35   0  43   6  23  13 141]]


In [17]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

                   Pop  Rock  Hip-Hop/Rap  Country  R&B/Soul  Metal  \
Pop                101    37           16       31        31     36   
Rock                23   108            1       33         9     50   
Hip-Hop/Rap          8     0          226        0        22      1   
Country             13    20            0      189         9      5   
R&B/Soul            19     9           30       20       125      8   
Metal                8    36            2        5         3    172   
Alternative/Indie   17    32           24       15        34     28   
Folk                20    35            0       43         6     23   

                   Alternative/Indie  Folk  
Pop                               14    26  
Rock                              23    30  
Hip-Hop/Rap                        5     0  
Country                            9    42  
R&B/Soul                          17     9  
Metal                             14    21  
Alternative/Indie                 88    25  


(8.0, 0.0)

## **Don't rerun the cell below, as this will result in adding dimensions to the sets, which will not work! Can rerun all other cells except this one.**

In [18]:
cnn_lyrics_train = train_lyrics[:, :, None]
cnn_lyrics_test = test_lyrics[:, :, None]

## Convolutional Neural Network

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPool2D, Dropout, Flatten

model = Sequential()
model.add(Conv1D(128, kernel_size=(1), activation='relu', input_shape=(len(word_list),1)))
model.add(Conv1D(16, (1), activation='relu'))
model.add(Dropout(0.25)) # Dropout 25% of the nodes of the previous layer during training
model.add(Flatten())     # Flatten, and add a fully connected layer
model.add(Dense(32, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 37818, 128)        256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 37818, 16)         2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 37818, 16)         0         
_________________________________________________________________
flatten (Flatten)            (None, 605088)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                19362848  
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

In [20]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(cnn_lyrics_train, train_genres, epochs=5, batch_size=16, validation_split=1/10)

Train on 11016 samples, validate on 1224 samples
Epoch 1/5
  192/11016 [..............................] - ETA: 5:01 - loss: 2.0776 - acc: 0.1406

KeyboardInterrupt: 

In [ ]:
results = model.evaluate(cnn_lyrics_test, test_genres, batch_size=16)